#### Import Required Packages

In [1]:
import os
import glob
import torch
from datasets import load_from_disk
from transformers import (
    PegasusForConditionalGeneration,
    PegasusTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

#### Set File Paths

In [2]:
TOKENIZED_TRAIN_PATH = r'..\..\Data\Processed\tokenized_pegasus_english\train'
TOKENIZED_TEST_PATH  = r'..\..\Data\Processed\tokenized_pegasus_english\test'
MODEL_OUTPUT_DIR     = r'..\..\Models\pegasus_english_finetuned'

#### Load Tokenized Datasets

In [3]:
train_dataset = load_from_disk(TOKENIZED_TRAIN_PATH)
test_dataset  = load_from_disk(TOKENIZED_TEST_PATH)
print(f"Train samples: {len(train_dataset)} | Test samples: {len(test_dataset)}")

Train samples: 28225 | Test samples: 2889


#### Initialize Model and Tokenizer

In [4]:
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Set Training Arguments

In [5]:
training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    eval_strategy="epoch",         # Only evaluates after each epoch
    save_strategy="steps",
    save_steps=500,
    logging_steps=100,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=2,                  # Keep only last 2 checkpoints
    load_best_model_at_end=False,
    logging_dir=os.path.join(MODEL_OUTPUT_DIR, "logs"),
    report_to="none"
)

#### Initialize Trainer

In [6]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

C:\Users\Abhi\AppData\Local\Temp\ipykernel_17376\2791099805.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


#### Resume from Latest Checkpoint If Exists

In [7]:
checkpoints = sorted(glob.glob(os.path.join(MODEL_OUTPUT_DIR, "checkpoint-*")))
if checkpoints:
    print(f"Resuming from checkpoint: {checkpoints[-1]}")
    trainer.train(resume_from_checkpoint=checkpoints[-1])
else:
    print("Starting training from scratch.")
    trainer.train()
print("Training complete.")


Resuming from checkpoint: ..\..\Models\pegasus_english_finetuned\checkpoint-500


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss
0,0.540100,0.768911


C:\Users\Abhi\.conda\envs\pegasusenv\lib\site-packages\transformers\modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training complete.


#### Save Final Model and Tokenizer

In [8]:
final_model_dir = os.path.join(MODEL_OUTPUT_DIR, "final_model")
trainer.save_model(final_model_dir)
tokenizer.save_pretrained(final_model_dir)
print(f"Final model and tokenizer saved to {final_model_dir}")

Final model and tokenizer saved to ..\..\Models\pegasus_english_finetuned\final_model
